In [1]:
import os
import platform
import sys

def get_project_root() -> str:
    system = platform.system()
    return r"D:\Users\tcshore\Documents\Projects\Tangrams\Data" if system == "Windows" else "/home/tshore/Projects/tangrams-restricted/Data"

infile_path = os.path.join(get_project_root(), "wordprobseqs-inflected-small.tsv")
print("Will read file \"{}\".".format(infile_path), file=sys.stderr)

Will read file "D:\Users\tcshore\Documents\Projects\Tangrams\Data\wordprobseqs-inflected-small.tsv".


In [2]:
import csv
import pandas as pd

RESULTS_FILE_CSV_DIALECT = csv.excel_tab
# NOTE: "category" dtype doesn't work with pandas-0.21.0 but does with pandas-0.21.1
__RESULTS_FILE_DTYPES = {"DYAD": "category", "WORD": "category", "IS_TARGET": bool, "IS_OOV": bool,
				 "IS_INSTRUCTOR": bool, "SHAPE": "category", "ONLY_INSTRUCTOR": bool, "WEIGHT_BY_FREQ": bool}

def read_results_file(inpath: str, encoding: str) -> pd.DataFrame:
	print("Reading \"{}\" using encoding \"{}\".".format(inpath, encoding), file=sys.stderr)
	result = pd.read_csv(inpath, dialect=RESULTS_FILE_CSV_DIALECT, sep=RESULTS_FILE_CSV_DIALECT.delimiter,
						 float_precision="round_trip",
						 encoding=encoding, memory_map=True, dtype=__RESULTS_FILE_DTYPES)
	return result

cv_results = read_results_file(infile_path, "windows-1252")
print("Read {} cross-validation results for {} dyad(s).".format(cv_results.shape[0], cv_results["DYAD"].nunique()),
      file=sys.stderr)

Reading "D:\Users\tcshore\Documents\Projects\Tangrams\Data\wordprobseqs-inflected-small.tsv" using encoding "windows-1252".
Read 180880 cross-validation results for 2 dyad(s).


In [3]:
def find_target_ref_rows(df: pd.DataFrame) -> pd.DataFrame:
	result = df.loc[df["IS_TARGET"] == True]
	result_row_count = result.shape[0]
	complement_row_count = df.loc[~df.index.isin(result.index)].shape[0]
	assert result_row_count + complement_row_count == df.shape[0]
	print("Found {} nontarget rows and {} target rows. Ratio: {}".format(complement_row_count, result_row_count,
																		 complement_row_count / float(
																			 result_row_count)), file=sys.stderr)
	return result

cv_results = find_target_ref_rows(cv_results)

Found 171836 nontarget rows and 9044 target rows. Ratio: 19.0


In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#import numpy as np

# Create vocab before splitting training and testing DFs so that the word feature set is stable
#vocab_words = tuple(sorted(cv_results["WORD"].unique()))
print("Fitting one-hot encoder for vocabulary of size {}.".format(len(cv_results["WORD"].unique())), file=sys.stderr)

# https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
# integer encode
label_encoder = LabelEncoder()
vocab_labels = label_encoder.fit_transform(cv_results["WORD"])
cv_results["WORD_LABEL"] = vocab_labels
#print(vocab_labels)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
vocab_labels = vocab_labels.reshape(len(vocab_labels), 1)
onehot_encoder.fit(vocab_labels)
#assert onehot_encoder.n_values_ == len(vocab_words)
#vocab_onehot_encoded = onehot_encoder.fit_transform(vocab_labels)
#print(vocab_onehot_encoded)
# invert first example
#inverted = label_encoder.inverse_transform([np.argmax(vocab_onehot_encoded[0, :])])
#print(inverted)

Fitting one-hot encoder for vocabulary of size 299.


OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=False)

In [5]:
from typing import Tuple

import numpy as np

def split_training_testing(df: pd.DataFrame, test_set_size : int) -> Tuple[pd.DataFrame, pd.DataFrame]:
	dyad_ids = df["DYAD"].unique()
	training_set_size = len(dyad_ids) - test_set_size
	if training_set_size < 1:
		raise ValueError("Desired test set size is {} but only {} dyads found.".format(test_set_size, len(dyad_ids)))
	else:
		training_set_dyads = frozenset(np.random.choice(dyad_ids, training_set_size))
		print("Training set dyads: {}".format(sorted(training_set_dyads)), file=sys.stderr)
		training_set_idxs = df["DYAD"].isin(training_set_dyads)
		training_set = df.loc[training_set_idxs]
		test_set = df.loc[~training_set_idxs]
		test_set_dyads = frozenset(test_set["DYAD"].unique())
		print("Test set dyads: {}".format(sorted(test_set_dyads)), file=sys.stderr)

		assert not frozenset(training_set["DYAD"].unique()).intersection(frozenset(test_set_dyads))
		return training_set, test_set
    
training_df, test_df = split_training_testing(cv_results, 1)
training_df = training_df.copy(deep=False)
test_df = test_df.copy(deep=False)

Training set dyads: ['1']
Test set dyads: ['17']


In [6]:
from typing import Iterator, List, Mapping

class SequenceMatrixFactory(object):

	def __init__(self, label_encoder, onehot_encoder):
		self.label_encoder = label_encoder
		self.onehot_encoder = onehot_encoder
    
	@property
	def feature_count(self):
		word_features = self.onehot_encoder.n_values_
		return word_features + 3

	def create_datapoint_feature_array(self, row: pd.Series) -> List[float]:
		#word_features = [0.0] * len(self.__vocab_idxs)
		# The features representing each individual vocabulary word are at the beginning of the feature vector
		#word_features[self.__vocab_idxs[row["WORD"]]] = 1.0
		#word_label = self.label_encoder.transform(row["WORD"])
		word_label = row["WORD_LABEL"]
		#print("Word label: {}".format(word_label), file=sys.stderr)
        # "OneHotEncoder.transform(..)" returns a matrix even if only a single value is passed to it, so get just the first (and only) row
		word_features = self.onehot_encoder.transform(word_label)[0]
		#print("Word features: {}".format(word_features), file=sys.stderr)
		# The word label for the one-hot encoding is that with the same index as the column that has a "1" value, i.e. the highest value in the vector of one-hot encoding values
		#inverse_label = np.argmax(word_features)
		#assert inverse_label == word_label
		#inverse_word = self.label_encoder.inverse_transform([inverse_label])
		#print("Inverse word label: {}".format(inverse_label), file=sys.stderr)
		is_instructor = 1.0 if row["IS_INSTRUCTOR"] else 0.0
		is_oov = 1.0 if row["IS_OOV"] else 0.0
		#is_target = 1.0 if row["IS_TARGET"] else 0.0
		score = row["PROBABILITY"]
		other_features = np.array((is_instructor, is_oov, score))
		#result = word_features + other_features
		result = np.concatenate((word_features, other_features))
		#print("Created a vector of {} features.".format(len(result)), file=sys.stderr)
		return result

	def __call__(self, df : pd.DataFrame) -> np.array:
		# https://stackoverflow.com/a/47815400/1391325
		df.sort_values("TOKEN_SEQ_ORDINALITY", inplace=True)
		sequence_groups = df.groupby(("CROSS_VALIDATION_ITER", "DYAD", "SPLIT_SEQ_NO", "UTT_START_TIME", "UTT_END_TIME", "ENTITY"), as_index=False)
		return np.array(tuple(tuple(self.__create_feature_vectors(seq)) for _, seq in sequence_groups))
        
	def __create_feature_vectors(self, df : pd.DataFrame) -> Iterator[List[float]]:
		# noinspection PyProtectedMember
		return (self.create_datapoint_feature_array(row._asdict()) for row in df.itertuples(index=False))

print("Splitting token sequences.", file=sys.stderr)
seq_matrix_factory = SequenceMatrixFactory(label_encoder, onehot_encoder)
training_matrix = seq_matrix_factory(training_df)
print("Created a training data matrix of shape {}.".format(training_matrix.shape), file=sys.stderr)
test_matrix = seq_matrix_factory(test_df)
print("Created a test data matrix of shape {}.".format(test_matrix.shape), file=sys.stderr)

Splitting token sequences.
Created a training data matrix of shape (1087, 4, 302).
Created a test data matrix of shape (1174, 4, 302).


In [7]:
training_x = training_matrix[:,:,:-1]
print(training_x.shape)
assert len(training_x.shape) == 3
training_y = training_matrix[:,:,-1]
print(training_y.shape)
assert len(training_y.shape) == 2

from keras.layers import Dense
from keras.layers import LSTM
from keras.models import Sequential

model = Sequential()
#word_embeddings = Embedding(len(vocab), embedding_vector_length, input_length=max_review_length)
#model.add(word_embeddings)
# model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
# input shape is a pair of (timesteps, features) <https://stackoverflow.com/a/44583784/1391325>
input_shape = training_x.shape[1:]
print("Input shape: {}".format(input_shape), file=sys.stderr)
units = training_y.shape[1]
print("Units: {}".format(units), file=sys.stderr)
lstm = LSTM(input_shape=input_shape, units=units)
#lstm = LSTM(batch_input_shape = training_x.shape, stateful = True, units=len(training_y.shape))
model.add(lstm)
model.add(Dense(units, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

(1087, 4, 301)
(1087, 4)


Using TensorFlow backend.
Input shape: (4, 301)
Units: 4


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 4)                 4896      
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 20        
Total params: 4,916
Trainable params: 4,916
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
# train LSTM
#model.fit(x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)
training_history = model.fit(x=training_x, y=training_y, epochs=1, verbose=1)

# train LSTM <https://machinelearningmastery.com/memory-in-a-long-short-term-memory-network/>
#epochs = 5
#for i in range(epochs):
#    x = training_x[i]
#    x_oldshape = x.shape
#    x_newshape = (-1, x_oldshape[0], x_oldshape[1])
#    x = np.reshape(x, x_newshape)
#    #print("x.shape = {}".format(x.shape), file=sys.stderr)
#    y = training_y[i]
#    y_oldshape = y.shape
#   y_newshape = (-1, y_oldshape[0])
#    y = np.reshape(y, y_newshape)
#    #print("y.shape = {}".format(y.shape), file=sys.stderr)
#    model.fit(x, y, epochs=1, batch_size=1, verbose=1, shuffle=False)
#    model.reset_states()
    
    

Epoch 1/1
1087/1087 [==============================] - 1s 745us/step - loss: 0.6850 - acc: 0.1911


In [10]:
# test LSTM


test_x = test_matrix[:,:,:-1]
print(test_x.shape)
assert len(test_x.shape) == 3
test_y = test_matrix[:,:,-1]
print(test_y.shape)
assert len(test_y.shape) == 2

seq_predicted_values = model.predict(test_x, verbose=0)
model.reset_states()
#print("result.shape = {}".format(seq_predicted_values.shape), file=sys.stderr)
for i, tested_seq in enumerate(test_x):
    print("Tested sequence: {}".format(tested_seq))
    predicted_values = seq_predicted_values[i]
    assert tested_seq.shape[:-1] == predicted_values.shape
    #assert tested_seq.shape[:-1] == actual_values.shape
    print("Predicted values: {}".format(predicted_values))
    actual_values = test_y[i]
    assert predicted_values.shape == actual_values.shape
    print("Actual values: {}".format(actual_values))
	#print('X=%.1f y=%.1f, yhat=%.1f' % (seq1[i], seq1[i+1], result[i]))

(1174, 4, 301)
(1174, 4)
Tested sequence: [[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
Predicted values: [ 0.44979015  0.47326729  0.49746504  0.49330011]
Actual values: [ 0.          0.          0.          0.49167632]
Tested sequence: [[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
Predicted values: [ 0.43872207  0.45959979  0.5142175   0.49573222]
Actual values: [ 0.          0.          0.          0.44697456]
Tested sequence: [[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
Predicted values: [ 0.44471413  0.45842895  0.51916051  0.50378704]
Actual values: [ 0.          0.          0.          0.50712497]
Tested sequence: [[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1

result.shape = (1174, 4)


Predicted values: [ 0.44632953  0.45909119  0.51649129  0.49990848]
Actual values: [ 0.          0.          0.54587104  0.57119133]
Tested sequence: [[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
Predicted values: [ 0.45229259  0.47189242  0.50705087  0.49604231]
Actual values: [ 0.          0.          0.54296002  0.48698339]
Tested sequence: [[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
Predicted values: [ 0.44495073  0.47131464  0.49858642  0.48746452]
Actual values: [ 0.          0.          0.50107075  0.47166645]
Tested sequence: [[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
Predicted values: [ 0.45229259  0.47189242  0.50705087  0.49604231]
Actual values: [ 0.          0.          0.49771107  0.63697675]
Tested sequence: [[ 0.  0.  0.

 [ 0.  0.  0. ...,  0.  1.  0.]]
Predicted values: [ 0.4673413   0.49213672  0.48972008  0.48687658]
Actual values: [ 0.44661592  0.51498776  0.51498776  0.48069197]
Tested sequence: [[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  1. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
Predicted values: [ 0.44721517  0.47334179  0.49546227  0.48303944]
Actual values: [ 0.50946684  0.49606882  0.48290374  0.7563622 ]
Tested sequence: [[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
Predicted values: [ 0.4621788   0.49037546  0.48468086  0.4833    ]
Actual values: [  5.19775017e-01   4.76215468e-01   1.52477519e-12   5.19775017e-01]
Tested sequence: [[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
Predicted values: [ 0.46714219  0.47244641  0.50897491  0.50074542]
Actual values: [  4.19810036e-07   4.08873

Actual values: [ 0.51898617  0.5543128   0.63450736  1.        ]
Tested sequence: [[ 0.  0.  1. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
Predicted values: [ 0.46440458  0.4721387   0.50819206  0.49973187]
Actual values: [ 0.50082359  0.65746366  0.48432361  0.48432361]
Tested sequence: [[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  1. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
Predicted values: [ 0.45290953  0.48032218  0.48615667  0.48070928]
Actual values: [ 0.51535395  0.51406079  0.7974699   0.51535395]
Tested sequence: [[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  1.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
Predicted values: [ 0.44350326  0.46091607  0.51951355  0.49806762]
Actual values: [ 0.65327535  0.66565381  0.91620061  0.56105572]
Tested sequence: [[ 0.  0.  1. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...

 [ 0.  0.  0. ...,  0.  1.  0.]]
Predicted values: [ 0.43573102  0.46536088  0.51183122  0.49220821]
Actual values: [ 0.45737033  0.41854398  0.6202752   0.32065514]
Tested sequence: [[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  1.]]
Predicted values: [ 0.45885986  0.460311    0.5251348   0.50368953]
Actual values: [ 0.48432361  0.48432361  0.46740208  0.49216561]
Tested sequence: [[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
Predicted values: [ 0.46644393  0.47883856  0.50053179  0.49423325]
Actual values: [ 0.7974699   0.51535395  0.4847682   0.50043706]
Tested sequence: [[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
Predicted values: [ 0.44958496  0.46284309  0.52310789  0.49990413]
Actual values: [ 0.91620061  0.56105572  0.67151917  0.9162006

 [ 0.  0.  1. ...,  0.  1.  0.]]
Predicted values: [ 0.45108816  0.47218478  0.50511044  0.49026245]
Actual values: [ 0.65605557  0.51558216  0.51876329  0.533209  ]
Tested sequence: [[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
Predicted values: [ 0.45652026  0.46949777  0.5120607   0.49512249]
Actual values: [  5.20044672e-01   3.33752639e-01   5.72810597e-01   4.88689288e-14]
Tested sequence: [[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
Predicted values: [ 0.43851459  0.46352652  0.50247121  0.48570663]
Actual values: [ 0.53863975  0.67354135  0.5226721   0.55950917]
Tested sequence: [[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
Predicted values: [ 0.4326013   0.46354312  0.51010501  0.49227428]
Actual values: [ 0.44600569  0.52771301  0

Actual values: [ 0.47205117  0.61863894  0.57122344  0.49858372]
Tested sequence: [[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
Predicted values: [ 0.45496652  0.4704949   0.50095272  0.49425358]
Actual values: [  2.66451842e-01   7.81390275e-01   4.96747893e-01   4.12686399e-53]
Tested sequence: [[ 0.  0.  1. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
Predicted values: [ 0.45073581  0.46494451  0.51218665  0.49671987]
Actual values: [ 0.533209    0.65574753  0.56799046  0.54327787]
Tested sequence: [[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
Predicted values: [ 0.45097765  0.46545428  0.50898927  0.49337718]
Actual values: [ 0.55950917  0.51896345  0.40370218  0.82912749]
Tested sequence: [[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0

Tested sequence: [[ 0.  0.  1. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
Predicted values: [ 0.45550889  0.46873778  0.5088346   0.49412698]
Actual values: [  4.82488414e-01   4.79129210e-01   7.39382771e-52   5.11950382e-01]
Tested sequence: [[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
Predicted values: [ 0.45268691  0.46712029  0.51392508  0.49713475]
Actual values: [ 0.82242038  0.68985738  0.31396617  0.65605557]
Tested sequence: [[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
Predicted values: [ 0.44489938  0.46658963  0.49881905  0.48730496]
Actual values: [ 0.51896345  0.10369947  0.08815231  0.07969663]
Tested sequence: [[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.

Predicted values: [ 0.45084432  0.46023589  0.52034569  0.50195998]
Actual values: [ 0.58004493  0.6348482   0.49348929  0.28250314]
Tested sequence: [[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
Predicted values: [ 0.45720434  0.47118536  0.50638044  0.49602973]
Actual values: [  6.34848198e-01   4.93489290e-01   2.82503142e-01   1.68984110e-07]
Tested sequence: [[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
Predicted values: [ 0.44333231  0.46570376  0.50332326  0.48906332]
Actual values: [  4.93489290e-01   2.82503142e-01   1.68984110e-07   4.05723461e-01]
Tested sequence: [[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
Predicted values: [ 0.44439965  0.47478259  0.49251592  0.48503351]
Actual values: [  2.82503142e-01   1.68984110e-07   4.0